# Talk to Your PDF: An Introduction to Retrieval Augmented Generation (RAG) for Google Colab

This notebook provides a hands-on introduction to the concept of Retrieval Augmented Generation (RAG). We'll build a simple application that allows you to "talk" to a PDF document by asking it questions in natural language. This is a powerful technique that combines the strengths of large language models (LLMs) with the ability to retrieve information from your own documents.

## What is RAG?

RAG is a technique that enhances the capabilities of LLMs by allowing them to access and incorporate information from external knowledge sources. Instead of relying solely on the knowledge the LLM was trained on, we first retrieve relevant information from a specific dataset (in our case, a PDF document) and then use that information to generate a more accurate and context-aware response.

### Why is RAG useful?

1.  **Reduces Hallucinations:** LLMs can sometimes generate plausible-sounding but incorrect or nonsensical information (known as "hallucinations"). RAG grounds the LLM's responses in a specific context, reducing the likelihood of such errors.
2.  **Uses Up-to-Date Information:** RAG allows LLMs to access information that wasn't part of their original training data, making it possible to work with real-time or proprietary data.
3.  **Increases Trust and Verifiability:** Because the generated responses are based on retrieved documents, users can verify the information and trace it back to the source.

## Setup and Dependencies

First, let's install the necessary libraries. We'll be using:

*   `google-generativeai`: To interact with the Gemini API.
*   `pypdf`: To read and extract text from PDF files.
*   `langchain`: A framework for developing applications powered by language models.
*   `faiss-cpu`: For efficient similarity search in our vector store.

In [ ]:
!pip install -q -U google-generativeai pypdf langchain faiss-cpu

### Configure your Gemini API Key

To use the Gemini API, you'll need an API key. If you don't have one, you can get it from the Google AI Studio: [https://aistudio.google.com/app/apikey](https://aistudio.google.com/app/apikey). Once you have your key, you can add it to the Colab secrets manager.

In [ ]:
import google.generativeai as genai
from google.colab import userdata
from getpass import getpass

try:
    # Use 'GEMINI_API_KEY' as the secret name
    api_key = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=api_key)
except userdata.SecretNotFoundError as e:
    print('GEMINI_API_KEY not found in Colab secrets. Please create it.')
    api_key = getpass('Or, enter your Gemini API key: ')
    genai.configure(api_key=api_key)

## Upload your PDF

In [ ]:
from google.colab import files

print('Please upload your PDF file')
uploaded = files.upload()

if len(uploaded.keys()) > 0:
    pdf_path = list(uploaded.keys())[0]

## Loading and Processing the PDF

Now, let's load the PDF file you uploaded and extract its text.

In [ ]:
from langchain.document.loaders import PyPDFLoader

if 'pdf_path' in locals():
    try:
        loader = PyPDFLoader(pdf_path)
        pages = loader.load_and_split()
        print(f"Successfully loaded {len(pages)} pages from the PDF.")
    except Exception as e:
        print(f"Error loading PDF: {e}")

## Creating a Vector Store

Next, we'll create a vector store from the extracted text. A vector store is a database that stores text as numerical representations (embeddings). This allows us to perform efficient similarity searches to find the most relevant text chunks for a given query.

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import GoogleGenerativeAiEmbeddings

if 'pages' in locals():
    embeddings = GoogleGenerativeAiEmbeddings(model="models/embedding-001")
    vector_store = FAISS.from_documents(pages, embeddings)
    print("Vector store created successfully.")

## Building the RAG Chain

Now, let's build the RAG chain. The chain will:

1.  Take a user's question.
2.  Retrieve relevant documents from the vector store.
3.  Pass the question and the retrieved documents to the Gemini model.
4.  Generate a response based on the provided context.

In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import GoogleGenerativeAI

if 'vector_store' in locals():
    llm = GoogleGenerativeAI(model="gemini-pro", temperature=0.3)
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vector_store.as_retriever(),
        return_source_documents=True
    )
    print("RAG chain created successfully.")

## Asking Questions

Now it's time to ask some questions to your PDF! Try asking questions that can be answered from the content of your document.

In [ ]:
if 'qa_chain' in locals():
    question = "What is the main topic of the document?"
    result = qa_chain({"query": question})
    print("Question:", result["query"])
    print("Answer:", result["result"])
    print("Source Documents:", [doc.metadata.get("page", "N/A") for doc in result["source_documents"]])

In [ ]:
if 'qa_chain' in locals():
    # Try another question
    question = "Summarize the key findings."
    result = qa_chain({"query": question})
    print("Question:", result["query"])
    print("Answer:", result["result"])
    print("Source Documents:", [doc.metadata.get("page", "N/A") for doc in result["source_documents"]])

## Conclusion

Congratulations! You've successfully built a RAG application that allows you to chat with your PDF. This is a powerful technique that can be applied to a wide range of applications, from customer support chatbots to research assistants. Feel free to experiment with different PDFs and questions to explore the capabilities of RAG.